<!-- dom:TITLE: Linear algebra, signal processing, and wavelets. A unified approach. Python notebooks -->
# Linear algebra, signal processing, and wavelets. A unified approach. Python notebooks
<!-- dom:AUTHOR: Øyvind Ryan -->
<!-- Author: --> **Øyvind Ryan**

Date: **Feb 17, 2016**

In [1]:
%matplotlib inline

import sys
sys.path.insert(0, 'code/python')
from numpy import *
from sound import*
import matplotlib.pyplot as plt
from time import *
from fft import *

In [2]:
from numpy import *
from sound import *

antsec = 3
fs = 44100
f = 440

Creating the samples of a pure tone:

In [3]:
t = linspace(0, antsec, fs*antsec)
x = sin(2*pi*f*t)

Listening to the pure tone and writing the samples to file

In [4]:
play(x, fs)
audiowrite('puretone440.wav', x, fs)

Listening to the square wave

In [5]:
samplesperperiod = fs/f
oneperiod = hstack([ones((samplesperperiod/2),dtype=float), \
                    -ones((samplesperperiod/2),dtype=float)])
x = tile(oneperiod, antsec*f)
play(x, fs)

Listening to the triangle wave

In [6]:
oneperiod = hstack([linspace(-1, 1, samplesperperiod/2), \
                    linspace(1, -1, samplesperperiod/2)])
x = tile(oneperiod, antsec*f)
play(x, fs)

Reading from file

In [7]:
x, fs = audioread('sounds/castanets.wav')

Listen to sound from the second channel

In [8]:
play(x[:, 1], fs);

Listen to the sample audio file at normal sampling rate,

In [9]:
play(x, fs)

double sampling rate,

In [10]:
play(x, 2*fs)

and half sampling rate

In [11]:
play(x, fs/2)

Play the sound backwards

In [12]:
N = shape(x)[0]
z = x[(N-1)::(-1), :]
play(z, fs)

Adding noise

In [13]:
c = 0.1
z = x + c*(2*random.random(shape(x))-1)
z /= abs(z).max()

play(z, fs)

## Exercise 2.1

In [14]:
t1 = arange(0, 4/440.0, 1/float(fs))
t2 = arange(4/440.0, 12/440.0, 1/float(fs))
t3 = arange(12/440.0, 20/440.0, 1/float(fs))

f1 = 0*t1                                # The first part of f
f2 = 2*((440*t2-4)/8)*sin(2*pi*440*t2)   # The second part of f
f3 = 2*sin(2*pi*440*t3)                  # The third part of f
x = hstack([f1, f2, f3])
x /= abs(x).max()
play(x, fs)

## Exercise 2.2

In [15]:
t = arange(0,3,1/float(fs))
x = sin(2*pi*440*t) + 0.3*sin(2*pi*4400*t)
x /= abs(x).max()
play(x, fs)

## Exercise 2.3

In [16]:
play_pure_sound(440)

In [17]:
play_pure_sound(1500)

## Exercise 2.4

In [18]:
play_square(1/440.0)

In [19]:
play_triangle(1/440.0)

## Exercise 2.5

In [20]:
play_square_fourier(1/440.0, 9)

In [21]:
play_triangle_fourier(1/440.0, 9)

## Exercise 2.6

In [22]:
x, fs = audioread('sounds/castanets.wav')
play_with_different_fs(x, fs)

## Exercise 2.7

In [23]:
play_reverse(x, fs)

## Exercise 2.8

In [24]:
play_with_noise( x, fs, 0.1)

## Example 2.27

The main code in `playDFT()`:

In [25]:
L = 10000
N = shape(x)[0]
# Zero out higher frequencies
y = fft.fft(x, None, 0)
y[(L+1):(N-L)] = 0;
newx = fft.ifft(y)

In [26]:
playDFT(7, True)

In [27]:
playDFT(3, True)

In [28]:
playDFT(7, False)

In [29]:
playDFT(3, False)

## Example 2.28

The main code in `playDFTthreshold()`:

In [30]:
threshold = 50
y = fft.fft(x, None, 0)
y = (abs(y) >= threshold)*y
newx = fft.ifft(y)

In [31]:
playDFTthreshold(0.02)

In [32]:
playDFTthreshold(0.1)

## Example 2.29

In [33]:
n = 5
y = fft.fft(x, None, 0)
y *= 2**n
y = round_(y)
y /= float(2**n)
newx = fft.ifft(y)

## Exercise 2.19

In [34]:
x = x[0:2**17]
y = fft.fft(x, None, 0)
y[(2**17/4):(3*2**17/4)] = 0
newx = abs(fft.ifft(y))
newx /= abs(newx).max()
play(newx, fs)

## Exercise 2.23

In [35]:
x0, fs = audioread('sounds/castanets.wav')

kvals = arange(3,16)
slowtime = zeros(len(kvals))
fasttime = zeros(len(kvals))
fastesttime = zeros(len(kvals))
N = 2**kvals
for k in kvals:
    x = x0[0:2**k].astype(complex)
    
    start = time()
    DFTImpl(x)
    slowtime[k - kvals[0]] = time() - start
    
    start = time()
    FFTImpl(x, FFTKernelStandard)
    fasttime[k - kvals[0]] = time() - start

    start = time()
    fft.fft(x, None, 0)
    fastesttime[k - kvals[0]] = time() - start

# a.
plt.plot(kvals, slowtime, 'ro-', \
     kvals, fasttime, 'go-', \
     kvals, fastesttime, 'bo-')
plt.grid('on')
plt.title('time usage of the DFT methods')
plt.legend(['DFT', 'Standard FFT', 'Built-in FFT'])
plt.xlabel('log_2 N')
plt.ylabel('time used [s]')
plt.show()
 
plt.figure()

# b.
plt.loglog(N, slowtime, 'ro-', N, fasttime, 'go-', N, fastesttime, 'bo-')
plt.axis('equal')
plt.legend(['DFT', 'Standard FFT', 'Built-in FFT'])
plt.show()

## Exercise 2.28

In [36]:
x0, fs = audioread('sounds/castanets.wav') 

kvals = arange(3,16)
slowtime = zeros(len(kvals))
fasttime = zeros(len(kvals))
fastesttime = zeros(len(kvals))
N = 2**kvals
for k in kvals:
    x = x0[0:2**k].astype(complex)
    
    start = time()
    FFTImpl(x, FFTKernelStandard)
    slowtime[k - kvals[0]] = time() - start
    
    start = time()
    FFTImpl(x, FFTKernelNonrec)
    fasttime[k - kvals[0]] = time() - start

    start = time()
    FFTImpl(x, FFTKernelSplitradix)
    fastesttime[k - kvals[0]] = time() - start

plt.plot(kvals, slowtime, 'ro-', \
     kvals, fasttime, 'bo-', \
     kvals, fastesttime, 'go-')
plt.grid('on')
plt.title('time usage of the DFT methods')
plt.legend(['Standard FFT algorithm', \
       'Non-recursive FFT', \
       'Split radix FFT'])
plt.xlabel('log_2 N')
plt.ylabel('time used [s]')
plt.show()

In [37]:
x, fs = audioread('sounds/castanets.wav')

## Exercise 3.23

In [38]:
play_with_echo(x, fs, 0.5, 20000);

## Exercise 3.25

In [39]:
play_with_reduced_treble(x, fs, 32)

In [40]:
play_with_reduced_bass(x, fs, 2);